# 2. Modelos de IA

Para o Sent.AI, três modelos de deep learning serão criados, treinados e avaliados - um para classificar se o texto exibe uma recomendação acionável, um para predizer o sentimento geral do texto e outro para identificar a emoção predominante no texto.

**Estrutura**

* Modelo de Recomendação
    * Carregamento dos Dados Preprocessados
    * Desenvolvimento da Arquitetura
    * Train Test Split
    * Treinamento do Modelo
    * Avaliação do Modelo
    * Serialização do Modelo
    
* Modelo de Sentimento Geral
    * Carregamento dos Dados Preprocessados
    * Desenvolvimento da Arquitetura
    * Train Test Split
    * Treinamento do Modelo
    * Avaliação do Modelo
    * Serialização do Modelo
    
* Modelo de Emoção Predominante
    * Carregamento dos Dados Preprocessados
    * Desenvolvimento da Arquitetura
    * Train Test Split
    * Treinamento do Modelo
    * Avaliação do Modelo
    * Serialização do Modelo

## Configurações iniciais

In [51]:
# Importações de libraries importantes

import pandas as pd
import numpy as np
import pickle as pl
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow.keras as tf_keras
import tensorflow as tf

## Modelo de Recomendação

### Carregamento dos Dados Preprocessados

In [4]:
# Carregamento dos dados

recomendacao_texto = pl.load(open("../dados/preprocessados/recomendacao_texto.pkl", "rb"))
recomendacao_label = pl.load(open("../dados/preprocessados/recomendacao_label.pkl", "rb"))

In [5]:
recomendacao_texto

array([[  17, 4125,   28, ...,    0,    0,    0],
       [  53,  420, 1166, ...,    0,    0,    0],
       [  41,  655, 2666, ...,    0,    0,    0],
       ...,
       [ 906,   27,  110, ...,    0,    0,    0],
       [  14, 7056, 6837, ...,    0,    0,    0],
       [  24,  834, 1433, ...,    0,    0,    0]])

In [6]:
recomendacao_label

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [7]:
# One-Hot Encoding da label

recomendacao_label = keras.utils.to_categorical(recomendacao_label, 2)
recomendacao_label

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

### Desenvolvimento da Arquitetura

In [74]:
class Modelo_de_Recomendacao:
    
    def __init__(self, dim_embedding, num_neuronios, attention, tamanho_vocab, tamanho_frase):
        
        self.dim_embedding = dim_embedding
        self.num_neuronios = num_neuronios
        self.tamanho_vocab = tamanho_vocab
        self.tamanho_frase = tamanho_frase
        self.attention = attention
        
    def constroi_modelo(self):
        
        # Camadas de entrada
        inputs = tf_keras.layers.Input(shape=(self.tamanho_frase, ))
        embedding = tf_keras.layers.Embedding(self.tamanho_vocab, self.dim_embedding, input_length=self.tamanho_frase)(inputs)
        
        # Camadas escondidas
        gru = tf_keras.layers.Bidirectional(tf_keras.layers.GRU(self.num_neuronios, return_sequences=True))(embedding)
        
        # Mecanismo de Atenção
        attention_layer1 = tf_keras.layers.Dense(self.attention, activation='tanh')(gru)
        attention_layer2 = tf_keras.layers.Dense(1)(attention_layer1)
        attention_layer3 = tf_keras.layers.Activation('softmax')(attention_layer2)
        attention_output = tf_keras.layers.Dot(axes=1)([attention_layer3, gru])
        flatten = tf_keras.layers.Flatten()(attention_output)
        flatten = tf_keras.layers.RepeatVector(self.tamanho_frase)(flatten)
        concatenated_attention = tf_keras.layers.Concatenate()([gru, flatten])
        
        # Camada de saída
        dense1 = tf_keras.layers.Dense(self.num_neuronios, activation='relu')(concatenated_attention)
        dropout = tf_keras.layers.Dropout(0.1)(dense1)
        dense2 = tf_keras.layers.Dense(self.num_neuronios / 2, activation='relu')(dropout)
        flatten2 = tf_keras.layers.Flatten()(dense2)
        dense_output = tf_keras.layers.Dense(2, activation='softmax')(flatten2)
        
        # Modelo
        modelo = tf_keras.models.Model(inputs=inputs, outputs=dense_output)
        
        return modelo

In [75]:
# Sumário do Modelo

modelo = Modelo_de_Recomendacao(128, 128, 64, 100000, 1000) # Hiperparâmetros de exemplo
modelo = modelo.constroi_modelo()
modelo.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 1000)]       0           []                               
                                                                                                  
 embedding_21 (Embedding)       (None, 1000, 128)    12800000    ['input_22[0][0]']               
                                                                                                  
 bidirectional_21 (Bidirectiona  (None, 1000, 256)   198144      ['embedding_21[0][0]']           
 l)                                                                                               
                                                                                                  
 dense_66 (Dense)               (None, 1000, 64)     16448       ['bidirectional_21[0][0]'] 

### Train Test Split

In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(recomendacao_texto, recomendacao_label, 
                                                    test_size=.15, random_state=341)

In [77]:
X_train.shape

(40815, 1000)

In [78]:
Y_train.shape

(40815, 2)

### Treinamento do Modelo

In [79]:
# PARÂMETROS

params = {'tamanho_vocab': 255415, 'tamanho_frase': 1000, 'learning_rate': .001,
          'num_neuronios': 256, 'dim_embedding': 256, 'attention': 128}

modelo = Modelo_de_Recomendacao(params["dim_embedding"], params["num_neuronios"], 
                                params["attention"], params["tamanho_vocab"], params["tamanho_frase"])

modelo = modelo.constroi_modelo()

In [80]:
modelo.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 1000)]       0           []                               
                                                                                                  
 embedding_22 (Embedding)       (None, 1000, 256)    65386240    ['input_23[0][0]']               
                                                                                                  
 bidirectional_22 (Bidirectiona  (None, 1000, 512)   789504      ['embedding_22[0][0]']           
 l)                                                                                               
                                                                                                  
 dense_71 (Dense)               (None, 1000, 128)    65664       ['bidirectional_22[0][0]'] 

In [81]:
modelo.compile(loss='categorical_crossentropy', 
               optimizer=keras.optimizers.Adam(learning_rate=params["learning_rate"]), metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
model_checkpoint = keras.callbacks.ModelCheckpoint('../modelos/modelo_recomendacao.h5', monitor='val_accuracy', save_best_only=True)

# Tempo inicial
now = datetime.datetime.now()

# Treinamento do modelo
history = modelo.fit(X_train, Y_train, epochs=2, validation_data=(X_test, Y_test), 
                     batch_size=64, callbacks=[early_stopping, model_checkpoint])
# Tempo final
now2 = datetime.datetime.now()

time = now2 - now
print(f"O tempo que levou para treinar é de {time}")

Epoch 1/2
405/638 [==================>...........] - ETA: 8:54:39 - loss: 7.9888 - accuracy: 0.9043